In [1]:
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import torch
import scipy.sparse as sp
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.metrics import roc_auc_score, average_precision_score
import time
import pickle
import pandas as pd
%matplotlib inline

from net.utils import *
from net import utils_netgan as utils
import net.net as net

In [2]:
_A_obs, _X_obs, _z_obs = utils.load_npz('../data/cora_ml.npz')
_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1
lcc = utils.largest_connected_components(_A_obs)
_A_obs = _A_obs[lcc,:][:,lcc]
_N = _A_obs.shape[0]

val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=True)

train_graph = sp.coo_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1]))).tocsr()
assert (train_graph.toarray() == train_graph.toarray().T).all()

Selecting 1 largest connected components


In [77]:
def configuration_model(A, B=None, EO=None):
    """Given two graphs A and B with same amount of edges, generates new graph by keeping overlapping edges,
       and rewiring remaining edges such that degrees of nodes in A are preserved. Self-loops and multiple 
       edges are removed. If B is None, draws the percentage EO of edges from A."""
    configuration_graph = np.zeros_like(A)
    if B is not None:
        configuration_graph = A * B
    else:
        B = np.triu(A, k=1)
        B /= B.sum()
        nonzero_ixs = B.nonzero()
        edges_from_A = np.random.choice(a=len(nonzero_ixs[0]), size=int(EO * A.sum() / 2), replace=False, 
                                        p=B[nonzero_ixs])
        configuration_graph[nonzero_ixs[0][edges_from_A], nonzero_ixs[1][edges_from_A]] = 1
        configuration_graph = configuration_graph + configuration_graph.T
    degrees = (A.sum(axis=-1) - configuration_graph.sum(axis=-1)).astype(int)
    stubs = np.zeros(degrees.sum())
    counter = 0
    for i in degrees.nonzero()[0]:
        stubs[counter: counter+degrees[i]] = i * np.ones(degrees[i])
        counter += degrees[i]
    np.random.shuffle(stubs)
    stubs = stubs.reshape(-1, 2).astype(int)
    configuration_graph[stubs[:, 0], stubs[:, 1]] = 1
    configuration_graph[stubs[:, 1], stubs[:, 0]] = 1  
    np.fill_diagonal(configuration_graph, 0)
    return configuration_graph

def LR_adjacency(A, sample_size, k):
    """ Sample graphs from a FORGE approach to the adjacency matrix A with rank k."""  
    # Low-rank approximation
    u, s, vt = sp.linalg.svds(A, k=k, which='LM')
    A_LR = u @ np.diag(s) @ vt
    # Normalize by trunctuating
    scores_matrix = np.minimum(np.maximum(A_LR, 0), 1)
    # Generate multiple graphs from score matrix
    sampled_graphs = []
    for i in range(sample_size):
        sampled_graph = utils.graph_from_scores(sp.csr_matrix(scores_matrix), A.sum())
        sampled_graphs.append(sampled_graph)
    return sampled_graphs

def LR_transition(A, sample_size, k):
    """ Sample graphs from a FORGE approach to the transition matrix P=D^{-1}A with rank k."""  
    # Transform to transition matrix
    P = A / np.sum(A, axis=-1, keepdims=True)
    # Low-rank approximation
    u, s, vt = sp.linalg.svds(P, k=k, which='LM')
    P_LR = u @ np.diag(s) @ vt
    # Set in correct form of transition matrices: positive entries and row-normalized
    P_LR = np.maximum(P_LR, 0)
    P_LR = P_LR / np.sum(P_LR, axis=-1, keepdims=True)
    # Backtransform to same scores_matrix as our method
    scores_matrix = scores_matrix_from_transition_matrix(transition_matrix=P_LR,
                                                         symmetric=True)
    # Generate multiple graphs from score matrix
    sampled_graphs = []
    for i in range(sample_size):
        sampled_graph = utils.graph_from_scores(sp.csr_matrix(scores_matrix), A.sum())
        sampled_graphs.append(sampled_graph)
    return sampled_graphs

In [33]:
sampled_graphs = LR_adjacency(A=train_graph, sample_size=10, k=1650)

dicts_LR_adjacency = []
for graph in sampled_graphs:
    statistics = utils.compute_graph_statistics(graph)
    statistics['overlap'] = utils.edge_overlap(train_graph.toarray(), graph) / train_graph.sum()
    dicts_LR_adjacency.append(statistics)

In [78]:
sampled_graphs = LR_transition(A=train_graph.toarray(), sample_size=10, k=1650)

dicts_LR_transition = []
for graph in sampled_graphs:
    statistics = utils.compute_graph_statistics(graph)
    statistics['overlap'] = utils.edge_overlap(train_graph.toarray(), graph) / train_graph.sum()
    dicts_LR_transition.append(statistics)

In [23]:
def EO_dict_of_lists_from_dicts(EO_dicts):
    EO_dict_of_lists = {}
    for dict_of_statistics in EO_dicts:
        EO_dict_of_lists = update_dict_of_lists(EO_dict_of_lists, dict_of_statistics)
    return EO_dict_of_lists

def get_average_statistics(EO_dict_of_lists):
    dict_of_average_statistics = {}
    for key in EO_dict_of_lists.keys():
        values = np.array(EO_dict_of_lists[key])
        dict_of_average_statistics[key] = (values.mean(), values.std())
    return dict_of_average_statistics
    
def print_table_of_average_statistics(list_of_average_statistics, keys, method_names):
    formatted_list_of_average_statistics = []
    # Convert (mean, std) into 'mean \pm std'
    for average_statistics in list_of_average_statistics:
        if type(average_statistics[keys[0]])==tuple:
            formatted_statistics = {}
            for key in average_statistics.keys():
                mean, std = average_statistics[key]
                formatted_statistics[key] = f'{round(mean,3)} \u00B1 {round(std,3)}'
        else:
            formatted_statistics = average_statistics
        formatted_list_of_average_statistics.append(formatted_statistics)
    df = pd.DataFrame(formatted_list_of_average_statistics, 
                      index=method_names)
    return df[keys]

In [79]:
NetGAN_dict = {'d_max' : 233, 'assortativity' : -0.066, 'triangle_count' : 1588, 'power_law_exp' : 1.793,
               'clustering_coefficient' : '2.44e-3', 'wedge_count' : 86763, 'rel_edge_distr_entropy' : 0.954,
               'LCC' : 2807, 'claw_count' : '26e6', 'gini' : 0.42, 'overlap' : 0.52, 'cpl' : 5.2}
relevant_keys = ['d_max', 'assortativity', 'triangle_count', 'power_law_exp', 'clustering_coefficient',
                 'wedge_count', 'rel_edge_distr_entropy', 'LCC', 'claw_count', 'gini', 'overlap', 'cpl', 'spectral_gap']
statistics_train = utils.compute_graph_statistics(train_graph.toarray())

joint_dict_adj = EO_dict_of_lists_from_dicts(dicts_LR_adjacency)
joint_dict_trans = EO_dict_of_lists_from_dicts(dicts_LR_transition)
dict_of_average_statistics_adj = get_average_statistics(joint_dict_adj)
dict_of_average_statistics_trans = get_average_statistics(joint_dict_trans)
print_table_of_average_statistics(list_of_average_statistics=[statistics_train,
                                                              dict_of_average_statistics_adj,
                                                              dict_of_average_statistics_trans,
                                                              NetGAN_dict],
                                  keys=relevant_keys,
                                  method_names=['CORA-ML', 'Adjacency approximation', 'Transition approximation',
                                                'NetGAN (paper)'])

,d_max,assortativity,triangle_count,power_law_exp,clustering_coefficient,wedge_count,rel_edge_distr_entropy,LCC,claw_count,gini,overlap,cpl,spectral_gap
CORA-ML,238,-0.0762641,2802,1.85506,0.00277104,101747,0.940663,2810,3.03351e+06,0.481863,NaN,5.63001,0.00611441
Adjacency approximation,133.0 ± 9.94,-0.04 ± 0.003,441.4 ± 34.584,1.723 ± 0.002,0.003 ± 0.0,49341.4 ± 1501.66,0.975 ± 0.001,2804.6 ± 3.955,539947.3 ± 88815.891,0.324 ± 0.003,0.539 ± 0.008,5.155 ± 0.023,0.04 ± 0.018
Transition approximation,149.4 ± 6.621,-0.057 ± 0.004,545.9 ± 18.316,1.771 ± 0.002,0.002 ± 0.0,58972.8 ± 748.096,0.964 ± 0.0,2778.4 ± 8.8,782233.9 ± 74036.898,0.392 ± 0.002,0.576 ± 0.003,5.063 ± 0.026,0.029 ± 0.004
NetGAN (paper),233,-0.066,1588,1.793,2.44e-3,86763,0.954,2807,26e6,0.42,0.52,5.2,NaN
